In [16]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver

In [17]:
# Set up Splinter
browser = Browser('chrome')

In [23]:
# for i in range(1, 3):
#     url = f'https://www.canadiantire.ca/en/promotions/hot-sale/home-pet.html?page={i}'
#     browser.visit(url)
#     html =browser.html  # Use browser.page_source to get the HTML after the page has loaded
#     soup = BeautifulSoup(html, 'html.parser')
    
#     # Extract the data for this page
#     stars_numofreviews = soup.find_all('div', class_='bv_text')
#     nameofthegoods = soup.find_all('div', class_='nl-product-card__title nl-product--trim')
#     price = soup.find_all('div', class_='nl-price--charge')
#     savings = soup.find_all('div', class_='nl-price__save-story')

# Create a list to hold data from all pages
all_page_data = []

for i in range(1):
    url = f'https://www.canadiantire.ca/en/promotions/hot-sale/home-pet.html?page={i}'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the data for this page
    nameofthegoods = soup.find_all('div', class_='nl-product-card__title nl-product--trim')
    price = soup.find_all('div', class_='nl-price--charge')
    savings = soup.find_all('div', class_='nl-price__save-story')
    stars = soup.find_all(class_='bv_averageRating_component_container')
    observations = soup.find_all(class_='bv_numReviews_component_container')


In [44]:
# Extract the star ratings from each element
observations_list = [obs.find(class_='bv_text').text.strip() for obs in observations]
stars_list = [star.find(class_='bv_text').text.strip() for star in stars]
names_list = [item.text.strip() for item in nameofthegoods]
price_list = [item.text.strip() for item in price]
savings_list = [item.text.strip() for item in savings]


In [46]:
    # Create a dictionary to hold the data for this page
    page_data_dict = {
        'Names': names_list,
        'Price': price_list,
        'Savings': savings_list,
        'Stars': stars_list,
        'Observations':observations_list
    }
    # Append the data for this page to the all_page_data list
    all_page_data.append(page_data_dict)

In [48]:
all_page_data

[{'Names': ['Panasonic NN-SU64LB Countertop Microwave Genius Sensor Cooking, 1.3-cu.ft., Matte Black',
   'Mastervac AT18115P 4.0 Peak HP Poly Wet/Dry Shop Vacuum with Hose and Accessories, 18.9-L',
   'Bissell Proheat2x® Revolution™ Pet Carpet & Upholstery Deep Cleaner',
   'type A Restore Stackable Storage Box with Lid, 68-L, Green',
   'Keurig® K-Select Single Serve Coffee Maker, Black',
   'Sauder 2-Door Wardrobe/Armoire Clothes Storage Cabinet With Hanger Rod & Shelves, White',
   'Lagostina Artiste-Clad Copper Cookware Set & Oven Safe, 12-pc',
   'Chamberlain® C2405C 1/2-HP Chain Drive Garage Door Opener with Wi-Fi',
   'T-fal Viva Aluminum Frying Pan Non-stick, Dishwasher & Oven Safe, Black',
   'ClosetMaid Expandable Closet Organizer Kit with 3-Closet Rod, White, 12-in',
   'Graco Pack ‘n Play Travel Dome LX Playard, Ashland',
   'Miele Complete C3 Limited Edition Multi-Floor Canister Vacuum Cleaner',
   'For Living Wood Top Kitchen Utility Storage Cart/Island With Locking Whee

In [52]:
# # Assuming you have single values for stars, observations, price, and savings
# # Let's create lists with a single element for each variable
# stars_list = [stars]
# observations_list = [observations]
# price_list = [price]
# savings_list = [savings]

# # Create a list of tuples containing the necessary information for each item
# items_info = list(zip(nameofthegoods, stars_list, observations_list, price_list, savings_list))

# # Call the create_summary_list function with the items_info list
# summary_list = create_summary_list(items_info)

import pandas as pd

# Assuming names_list, price_list, savings_list, stars_list, and observations_list contain data for individual products

# Create a list of dictionaries where each dictionary represents a product
products_data = []
for i in range(len(names_list)):
    product_data = {
        'Names': names_list[i],
        'Price': price_list[i],
        'Savings': savings_list[i],
        'Stars': stars_list[i],
        'Observations': observations_list[i]
    }
    products_data.append(product_data)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(products_data)

# Display the DataFrame
df



,Names,Price,Savings,Stars,Observations
0,Panasonic NN-SU64LB Countertop Microwave Geniu...,$179.99,Save 10% ($30),4.3,(250)
1,Mastervac AT18115P 4.0 Peak HP Poly Wet/Dry Sh...,$59.99,1/3 Off (Save $30),4.1,(187)
2,Bissell Proheat2x® Revolution™ Pet Carpet & Up...,$249.99,1/2 Price (Save $250),4.3,(1866)
3,"type A Restore Stackable Storage Box with Lid,...",$7.99,1/2 Price,4.1,(238)
4,"Keurig® K-Select Single Serve Coffee Maker, Black",$139.99,Save 15% ($30),4.5,(477)
5,Sauder 2-Door Wardrobe/Armoire Clothes Storage...,$139.99,Save 35% ($90),3.3,(33)
6,Lagostina Artiste-Clad Copper Cookware Set & O...,$449.99,Save 75% ($1550),4.4,(76)
7,Chamberlain® C2405C 1/2-HP Chain Drive Garage ...,$279.99,Save 25% ($100),4.4,(125)
8,"T-fal Viva Aluminum Frying Pan Non-stick, Dish...",From $11.99,Save up to 70% (up to $50),4.5,(692)
9,ClosetMaid Expandable Closet Organizer Kit wit...,$95.99,Save 20% ($24),4.0,(118)


In [52]:
summary_list=create_summary_list(nameofthegoods, stars,observations, price, savings)
summary_list

TypeError: create_summary_list() takes 4 positional arguments but 5 were given

In [48]:
# Import Pandas
import pandas as pd

# Convert the articles list to a Pandas DataFrame
df = pd.DataFrame(summary_list)

# Display the first few rows using the head() function
df.head()





,Name,Stars,Observations,Price,Savings
0,MASTER Chef Energy Star Compact Refrigerator w...,4.4,None,$169.99,Save 30% ($80)


In [15]:
browser.quit()